## 1. Setup and Authentication

First, install the library and set up your API key.

In [ ]:
# Install the OpenAI library (run once)
# !pip install openai

from openai import OpenAI

OPENAI_API_KEY = 'sk-yourapikey'
client = OpenAI(api_key=OPENAI_API_KEY)

print("OpenAI client initialized")


OpenAI client initialized


## 2. Basic Chat Completion

The fundamental operation: sending a message and getting a response.

In [3]:
# Simple chat completion
response = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant specialized in graph theory."},
        {"role": "user", "content": "What is the shortest path algorithm called?"}
    ]
)

print("Response:")
print(response.choices[0].message.content)
print(f"\nTokens used: {response.usage.total_tokens}")
print(f"Model: {response.model}")

Response:
The shortest path algorithm is commonly known as Dijkstra's algorithm. It is used to find the shortest path from a starting node to all other nodes in a weighted graph with non-negative edge weights. Other notable shortest path algorithms include:

1. **Bellman-Ford Algorithm**: This algorithm can handle graphs with negative edge weights and can also detect negative weight cycles.
2. **A***: This is a heuristic-based algorithm that is often used for pathfinding and graph traversal, particularly in AI and gaming.
3. **Floyd-Warshall Algorithm**: This algorithm finds the shortest paths between all pairs of nodes and can handle negative weights as well.
4. **Bidirectional Dijkstra's Algorithm**: An optimized version of Dijkstra's algorithm that runs in both directions from the start and end nodes, which can be more efficient in certain cases.

Each of these algorithms is designed for specific types of graphs and use cases.

Tokens used: 216
Model: openai/gpt-4o-mini


## 3. Multi-turn Conversations

Building a conversation history - essential for the debate mechanism.

In [4]:
# Simulate a multi-turn debate
messages = [
    {"role": "system", "content": "You are a Proposer agent in a graph problem-solving system."},
    {"role": "user", "content": "Given a graph with nodes [1,2,3,4,5] and edges [(1,2),(2,3),(3,4),(4,5),(2,5)], find the shortest path from 1 to 5."}
]

# Proposer's first response
response = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=messages
)

proposer_response = response.choices[0].message.content
print("PROPOSER:")
print(proposer_response)

# Add to conversation history
messages.append({"role": "assistant", "content": proposer_response})

# Now add Verifier's feedback
messages.append({
    "role": "user",
    "content": "As a Verifier agent, check if this solution is correct. Verify each step."
})

# Verifier's response
response = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=messages
)

print("\nVERIFIER:")
print(response.choices[0].message.content)

print(f"\nTotal messages in conversation: {len(messages) + 1}")

PROPOSER:
To find the shortest path from node 1 to node 5 in the given graph with nodes [1, 2, 3, 4, 5] and edges [(1, 2), (2, 3), (3, 4), (4, 5), (2, 5)], we can use breadth-first search (BFS) since the graph is unweighted.

Here’s a step-by-step breakdown of the traversal:

1. Start from node 1.
2. From node 1, move to node 2 (the only connection).
3. From node 2, you can move to either node 3 or node 5. We have two paths to consider:
   - Path through node 3: 
     - From node 2 to node 3.
     - From node 3 to node 4.
     - From node 4 to node 5.
     - The path is 1 → 2 → 3 → 4 → 5.
   - Direct path:
     - From node 2 to node 5.
     - The path is 1 → 2 → 5.

4. Compare the lengths of both paths:
   - 1 → 2 → 5 has 2 edges (shorter).
   - 1 → 2 → 3 → 4 → 5 has 4 edges.

Thus, the shortest path from node 1 to node 5 is:

**1 → 2 → 5** 

This path has a total of 2 edges.

VERIFIER:
To verify the shortest path solution from node 1 to node 5 in the given graph, let's go through each

## 4. System Messages - Defining Agent Roles

Different system prompts create different agent behaviors.

In [5]:
# Define system prompts for each agent
PROPOSER_SYSTEM = """You are a Proposer agent in a multi-agent graph problem-solving system.
Your role is to suggest solutions to graph problems step by step.
Be creative but logical in your approach."""

VERIFIER_SYSTEM = """You are a Verifier agent in a multi-agent graph problem-solving system.
Your role is to critically examine proposed solutions and identify any errors.
Check for:
- Invalid moves (edges that don't exist)
- Repeated nodes (if not allowed)
- Logic errors
Be thorough and precise."""

COORDINATOR_SYSTEM = """You are a Coordinator agent in a multi-agent graph problem-solving system.
Your role is to:
1. Review proposals and verifications
2. Make final decisions
3. Decide if more debate rounds are needed
Be balanced and fair in your judgments."""

graph_problem = "Find the shortest path from node 1 to node 5 in a graph with edges: (1,2), (2,3), (3,5), (1,4), (4,5)."

# Proposer suggests a solution
proposer_response = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=[
        {"role": "system", "content": PROPOSER_SYSTEM},
        {"role": "user", "content": graph_problem}
    ],
)

print("=== PROPOSER ===")
print(proposer_response.choices[0].message.content)

# Verifier checks the proposal
verifier_response = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=[
        {"role": "system", "content": VERIFIER_SYSTEM},
        {"role": "user", "content": f"Problem: {graph_problem}\n\nProposed solution: {proposer_response.choices[0].message.content}"}
    ],
)

print("\n=== VERIFIER ===")
print(verifier_response.choices[0].message.content)

# Coordinator makes decision
coordinator_response = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=[
        {"role": "system", "content": COORDINATOR_SYSTEM},
        {"role": "user", "content": f"Problem: {graph_problem}\n\nProposal: {proposer_response.choices[0].message.content}\n\nVerification: {verifier_response.choices[0].message.content}"}
    ],
)

print("\n=== COORDINATOR ===")
print(coordinator_response.choices[0].message.content)

=== PROPOSER ===
To find the shortest path from node 1 to node 5 in the graph defined by the edges (1,2), (2,3), (3,5), (1,4), and (4,5), we can visualize or map out the graph first.

### Graph Representation
The graph can be represented as follows:

- **Nodes:** 1, 2, 3, 4, 5
- **Edges:**
  - 1 → 2
  - 2 → 3
  - 3 → 5
  - 1 → 4
  - 4 → 5

### Step-by-step Path Finding
Now, let’s evaluate the possible paths from node 1 to node 5.

1. **Path 1:**
   - Start at 1
   - Move to 2 (1 → 2)
   - Move to 3 (2 → 3)
   - Move to 5 (3 → 5)
   - Path: 1 → 2 → 3 → 5  
   - Total edges: 3

2. **Path 2:**
   - Start at 1
   - Move to 4 (1 → 4)
   - Move to 5 (4 → 5)
   - Path: 1 → 4 → 5  
   - Total edges: 2

### Comparing Paths
- **Path 1** has a total of 3 edges.
- **Path 2** has a total of 2 edges.

### Conclusion
The shortest path from node 1 to node 5 is **Path 2**, which is:

**1 → 4 → 5**

This path consists of 2 edges, making it the shortest one.

=== VERIFIER ===
Your proposed solution to fi